### Create Month and Year Information "globally"

In [ ]:
import datetime

# Get current month and year
now = datetime.datetime.now()
# Adjust for month and year, considering the shift back by one month
if now.month == 1:
    # If it's January, set month to December and year to the previous year
    month = '12'
    year = str(now.year - 1)[-2:]
else:
    # Otherwise, just subtract one from the month
    month = str(now.month - 1).zfill(2)
    year = str(now.year)[-2:]

### Importing the needed libraries

In [ ]:
import pandas as pd
import PyPDF2
import pyexcel as pe

### Revolut

Converting the Revolut's EUR transaction data into a DataFrame. I do not need to account separately for the fees as that will already be incorporated in the delta between the amount converted and the amount received

In [ ]:
# Path to CSV file with month and year placeholders
csv_path = r'C:\Users\filip\Desktop\Monthly Transactions\Revolut_{}{}.csv'.format(month, year)

# Read CSV file into a pandas DataFrame
rvlt = pd.read_csv(csv_path)

# convert the date_column to datetime objects
rvlt['Completed Date'] = pd.to_datetime(rvlt['Completed Date'])

# format the datetime objects into the desired truncated format
# I need to rename the column because it causes issues in the following SQL query
rvlt['Completed Date'] = rvlt['Completed Date'].dt.strftime('%Y-%m-%d')
rvlt.rename(columns={'Completed Date': 'Date','Type':'Transaction_Type'}, inplace=True)

# Using .loc to select only rows where 'State' is 'COMPLETED'
rvlt = rvlt.loc[rvlt['State'] == 'COMPLETED']

# Do something with the DataFrame, e.g.
print(rvlt.head())


Getting the Revolut's GBP statement as well

### Fideuram

The below appears to work for Fideuram while pd.read_excel was having issues

In [ ]:
# Path to XLSX file with month and year placeholders
xlsx_path = r'C:\Users\filip\Desktop\Monthly Transactions\Fideuram_{}{}.xlsx'.format(month, year)

# Read XLSX file into a pyexcel sheet object
sheet = pe.get_sheet(file_name=xlsx_path, name_columns_by_row=0, start_row=18)

# Get the data from the sheet as a list of lists and convert it to a pandas DataFrame
data = sheet.get_array()
headers = data[1]  # Assumes headers are in the second row
#print(headers)

fdrm = pd.DataFrame(data[2:], columns=headers)  # Assumes data starts from the third row
#print(fdrm)

# Drop rows with null values in the 'Data' column
fdrm = fdrm.dropna(subset=['Data'])

# Convert the date_column to datetime objects
fdrm['Data'] = pd.to_datetime(fdrm['Data'])

# Format the datetime objects into the desired truncated format
# I need to rename the column because it causes issues in the following SQL query
fdrm['Data'] = fdrm['Data'].dt.strftime('%Y-%m-%d')
fdrm.rename(columns={'Operazione': 'TRANSACTION_TYPE', 'Data': 'DATE', 'Dettagli': 'DESCRIPTION', 
'Categoria ': 'SUGGESTED_CATEGORY','Importo':'Amount','Valuta': 'CURRENCY', 'Contabilizzazione': 'STATE'}, inplace=True)

# Convert the 'Importo' column to float
fdrm['Amount'] = fdrm['Amount'].astype(float)

# Do something with the DataFrame, e.g.
print(fdrm)


### Trade Republic

For the purpose of the expenses spreadsheet I am only taking the Card Transactions

In [ ]:
#Adding the extended version of year which I will be using for the date construction once I start parsing the transaction list
if now.month == 1:
    # If it's January, set month to December and year to the previous year
    year_long = str(now.year - 1)
else:
    # Otherwise, just subtract one from the month
    year_long = str(now.year)

# Path to PDF file
pdf_path = r'C:\Users\filip\Desktop\Monthly Transactions\TradeRepublic_{}{}.pdf'.format(month, year)

# Open the PDF file in read binary mode
with open(pdf_path, 'rb') as pdf_file:
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    # Get the number of pages in the PDF document
    num_pages = len(pdf_reader.pages)
    # Initialize an empty list to hold the extracted data
    data = []
    
    
    # Loop through each page of the PDF document and extract the text
    for page_num in range(num_pages):
        # Get the current page object
        page = pdf_reader.pages[page_num]
        
        # Extract the text from the current page
        page_text = page.extract_text()

         # Split the text into lines
        lines = page_text.split('\n')
        
        # Determine if this is the first page
        first_page = (page_num == 0)
        
        # Skip the first 7 lines (rows)
        row_list=[]
        for line_num, line in enumerate(lines):
            print (line_num)
            print(line)
            #print(len(line))
            #If I have already found all information for a row (e.g. date, detail and amount) I reset the row_list
            #Otherwise I will not empty row_list since the information to complete the amount column will be added
            #in the next row
            if(len(row_list)==3):
                row_list=[] 
            else:
                pass
            #This is superflous
            if first_page and line_num < 9:
                continue
            
            # If this is the 8th row, use it as column names
            #if first_page and line_num == 7:
             #   columns = line.split()
                #debugging
              #  print(columns)
            else:
                #Given the new structure of the TR PDF if I find a row that is 7 characters long that will be 
                if len(line)==7:
                    row_list=[]
                    date_str=line[:6] + ' ' + year_long
                    print(date_str)
                    # Convert the string to a datetime object
                    date_obj = datetime.datetime.strptime(date_str, "%d %b %Y")

                    # Convert the datetime object to the desired format
                    date_col = date_obj.strftime("%d/%m/%Y")
                    
                    print(date_col)
                    row_list.append(date_col)
                    
                if "Transaction" in line:
                    #exception for JPN Exchange only - abroad payments may appear in four rows instead of three
                        
                    EUR_pos_first=line.find("€")
                    EUR_pos_last=line.rfind("€")
                    
                    if line.find("€") != -1:
                    
                    
                        Detail_col=line[len("Transaction"):EUR_pos_first]
                        print(Detail_col)
                        row_list.append(Detail_col)
                    
                        Amount_col=line[EUR_pos_first+1:EUR_pos_last-1]
                        print(Amount_col)
                        row_list.append(Amount_col)
                    
                    else:
                        Detail_col=line
                        # Get the next line
                        next_line = lines[line_num + 1]
                        EUR_pos_first=next_line.find("€")
                        EUR_pos_last=next_line.rfind("€")
                        Amount_col=next_line[EUR_pos_first+1:EUR_pos_last-1]
                        print(Amount_col)
                        row_list.append(Amount_col)
                
                if "Payment" in line:
                    
                    EUR_pos_first=line.find("€")
                    EUR_pos_last=line.rfind("€")
                    
                    
                    Detail_col=line[len("Payment"):EUR_pos_first]
                    print(Detail_col)
                    row_list.append(Detail_col)
                    
                    Amount_col=line[EUR_pos_first+1:EUR_pos_last-1]
                    print(Amount_col)
                    row_list.append(Amount_col)
                    
                if "Reward" in line:
                    
                    EUR_pos_first=line.find("€")
                    EUR_pos_last=line.rfind("€")
                    
                    
                    Detail_col=line[len("Reward")+4:EUR_pos_first]
                    print(Detail_col)
                    row_list.append(Detail_col)
                    
                    Amount_col=line[EUR_pos_first+1:EUR_pos_last-1]
                    print(Amount_col)
                    row_list.append(Amount_col)
                    
                if "Transfer" in line:
                    
                    #print("Wrong transfer format!!!!!")
                    
                    EUR_pos_first=line.find("€")
                    EUR_pos_last=line.rfind("€")
                    
                    #Adding the case in which the transfer description is too long and it reaches the next row
                    #ie the case in which the EUR sign is not found and the amount is in the next row (digits only)
                    
                    if line.find("€") != -1:
                    
                        Detail_col=line[len("Transfer")+4:EUR_pos_first]
                        print(Detail_col)
                        row_list.append(Detail_col)
                    
                        Amount_col=line[EUR_pos_first+1:EUR_pos_last-1]
                        print(Amount_col)
                        row_list.append(Amount_col)
                    
                    else:
                        #print("Wrong transfer format!!!!!")
                        #There are cases in which the 
                        Detail_col=line
                        row_list.append(Detail_col)
                        # Get the next line
                        next_line = lines[line_num + 1]
                        if next_line.strip().isdigit():
                        # This line has only digits
                            Amount_col=next_line
                            print(Amount_col)
                            row_list.append(Amount_col)
                        else:
                            # It has letters, symbols, or spaces
                            EUR_pos_first=next_line.find("€")
                            EUR_pos_last=next_line.rfind("€")
                            Amount_col=next_line[EUR_pos_first+1:EUR_pos_last-1]
                            print(Amount_col)
                            row_list.append(Amount_col)
                    
                else:
                    pass
                    
                #print(row_list)
                if(len(row_list)==3):
                    print(row_list)
                    data.append(row_list)
                    #print(data)
                else:
                    pass

# Set the display option to show all rows
pd.set_option('display.max_rows', None)

# Optional: Show all columns as well
pd.set_option('display.max_columns', None)                
# Create a pandas DataFrame from the data list and the column names
#print(data)
traderep = pd.DataFrame(data, columns=['Date','Detail','Amount'])

# convert the date_column to datetime objects
traderep['Date'] = pd.to_datetime(traderep['Date'], format='%d/%m/%Y')

# format the datetime objects into the desired truncated format
# I need to rename the column because it causes issues in the following SQL query
traderep['Date'] = traderep['Date'].dt.strftime('%Y-%m-%d')
# I strip the EUR sign from the Amount column
traderep['Amount'] = traderep['Amount'].str.replace('€', '')
traderep['Amount'] = traderep['Amount'].str.replace(',', '')
# Print your DataFrame

 
# Create a boolean mask to select rows where 'Detail' does not contain 'Reloading'
#The ~ operator is used to negate the boolean mask, selecting only the rows that do not contain "Reloading".
#Replacing Reloading with Amazon EU

# Create a boolean mask to select rows where 'Detail' does not contain any of the specified phrases
mask = ~traderep['Detail'].str.contains('Interest payment|Saveback|Deposit accepted', regex=True)

# Add a minus sign to the 'Amount' column only for rows matching the mask
traderep.loc[mask, 'Amount'] = '-' + traderep.loc[mask, 'Amount']
    
traderep['Amount'] = traderep['Amount'].astype(float)
    
traderep.rename(columns={'Detail': 'DESCRIPTION'}, inplace=True)

# Print the DataFrame
print(traderep)

### Pluxee/Sodexo 

In [ ]:
# Path to PDF file
pdf_path = r'C:\Users\filip\Desktop\Monthly Transactions\Sodexo_{}{}.pdf'.format(month, year)

# Open the PDF file in read binary mode
with open(pdf_path, 'rb') as pdf_file:
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    # Get the number of pages in the PDF document
    num_pages = len(pdf_reader.pages)
    # Initialize an empty list to hold the extracted data
    data = []
    
    
    # Loop through each page of the PDF document and extract the text
    for page_num in range(num_pages):
        # Get the current page object
        page = pdf_reader.pages[page_num]
        
        # Extract the text from the current page
        page_text = page.extract_text()

         # Split the text into lines
        lines = page_text.split('\n')
        
        # Determine if this is the first page
        first_page = (page_num == 0)
        
        # Skip the first 7 lines (rows)
        row_list=[]
        transaction_error=0
        for line_num, line in enumerate(lines):
            print (line_num)
            print(line)
            #If I have already found all information for a row (e.g. date, detail and amount) I reset the row_list
            #Otherwise I will not empty row_list since the information to complete the amount column will be added
            #in the next row
            if(len(row_list)==3):
                row_list=[] 
            #Initializing the transaction error to 0
                transaction_error=0
            else:
                pass
            if first_page and line_num < 7:
                continue
            
            # If this is the 8th row, use it as column names
            if first_page and line_num == 7:
                columns = line.split()
                #debugging
                print(columns)
            else:
                # Split the line into columns
                #If the line starts with P (e.g. Purchase row in the .PDF) I dont do anything
                #If the line is regular (e.g. it starts with a date), I append the date to the list
                #date has a fixed length of 20 characters
                if line.startswith("Purchase") or line.startswith("Reloading"):
                # Your code if line[:1]=="Purchase" :
                    pass
                else:
                    date_col=line[:20]
                    row_list.append(date_col)
                #I find where the EUR sign is positioned in the row
                EUR_pos=line.rfind("€")
                #EUR has a position of -1 if it is not present in the row (e.g. in all Purchase transactions)
                if EUR_pos==-1:
                    Detail_col=line[20:]
                    row_list.append(Detail_col)
                else:
                # Here we have to distinguish between the Purchase and Regular transactions
                #Are the regular transactions faulty transactions? It would appear so
                # i am adding a transaction error parameter that I will check alongside the length of the row list
                    if line.startswith("Purchase") or line.startswith("Reloading"):
                        Amount_col = line[EUR_pos:len(line)]
                        row_list.append(Amount_col)
                    else:
                        transaction_error=1
                        Detail_col=line[20:EUR_pos-1]
                        Amount_col=line[EUR_pos:len(line)]
                        row_list.append(Detail_col)
                        row_list.append(Amount_col)
                        
                # Add the row to the data list
                #print(row_list)
                #Neither row_list nor data are ever getting printed, which means that the le(row_list)==3 condition
                #is never met
                print(row_list)
                if(len(row_list)==3 and transaction_error==0):
                    #print(row_list)
                    data.append(row_list)
                    #print(data)
                else:
                    pass
                    
    # Create a pandas DataFrame from the data list and the column names
    #print(data)
    sodexo = pd.DataFrame(data, columns=columns)
    print(sodexo)
    # convert the date_column to datetime objects
    sodexo['Date'] = pd.to_datetime(sodexo['Date'])

    # format the datetime objects into the desired truncated format
    # I need to rename the column because it causes issues in the following SQL query
    sodexo['Date'] = sodexo['Date'].dt.strftime('%Y-%m-%d')
    # I strip the EUR sign from the Amount column
    sodexo['Amount'] = sodexo['Amount'].str.replace('€', '')
    
   # Create a boolean mask to select rows where 'Detail' does not contain 'Reloading'
   #The ~ operator is used to negate the boolean mask, selecting only the rows that do not contain "Reloading".
   #Replacing Reloading with Amazon EU
    mask = ~sodexo['Detail'].str.contains('Amazon EU')
    
    # Add a minus sign in front of the selected rows in the Amount column
    sodexo.loc[mask, 'Amount'] = '-' + sodexo.loc[mask, 'Amount']
    
    sodexo['Amount'] = sodexo['Amount'].astype(float)
    
    sodexo.rename(columns={'Detail': 'DESCRIPTION'}, inplace=True)

    # Print the DataFrame
    print(sodexo)

### Satispay

Changed the way the data is parsed on 06/08 due to a change in the statement's format

In [ ]:
import pandas as pd

# Path to CSV file
csv_path = r'C:\Users\filip\Desktop\Monthly Transactions\Satispay_{}{}.csv'.format(month, year)

try:
    # First, detect header row
    with open(csv_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        for i, line in enumerate(lines):
            if 'Amount' in line and 'Date' in line:
                header_row = i
                break
        else:
            raise ValueError("Could not find the header row with 'Amount' and 'Date'.")

    # Read the CSV starting from the correct row
    satispay = pd.read_csv(csv_path, skiprows=header_row)

    # Standardize column names to lowercase
    satispay.columns = satispay.columns.str.lower()

    # Parse 'date' with flexible datetime parsing
    satispay['date'] = pd.to_datetime(satispay['date'], dayfirst=True)

    # Format to 'YYYY-MM-DD'
    satispay['date'] = satispay['date'].dt.strftime('%Y-%m-%d')

    # Rename 'extra info' to 'Extra_Info' (if it exists)
    if 'extra info' in satispay.columns:
        satispay.rename(columns={'extra info': 'Extra_Info'}, inplace=True)

    # Concatenate extra info to name if needed
    if 'extra_info' in satispay.columns and 'name' in satispay.columns:
        satispay['name'] = satispay.apply(
            lambda row: row['name'] + ' ' + row['extra_info'] if pd.notnull(row['extra_info']) else row['name'],
            axis=1
        )

    # Clean and convert amount column
    satispay['amount'] = satispay['amount'].astype(str).str.replace(',', '.')
    satispay['amount'] = satispay['amount'].astype(float)

    # Final renames
    satispay.rename(columns={'kind': 'TRANSACTION_TYPE', 'name': 'DESCRIPTION'}, inplace=True)

    print(satispay.head())

except FileNotFoundError:
    print(f"No CSV file found at {csv_path}")
except Exception as e:
    print("An error occurred:", e)


### BGL BNP

In [ ]:
# Path to Excel file with month and year placeholders
xlsx_path = r'C:\Users\filip\Desktop\Monthly Transactions\BGLBNP_{}{}.xlsx'.format(month, year)

# Read Excel file into a pandas DataFrame
BGLBNP = pd.read_excel(xlsx_path, engine='openpyxl', usecols='A:E')
# Close the Excel file
# convert the date_column to datetime objects
BGLBNP['Posting Date']= pd.to_datetime(BGLBNP['Date comptable'])

# format the datetime objects into the desired truncated format
# I need to rename the column because it causes issues in the following SQL query
BGLBNP['Posting Date'] = BGLBNP['Posting Date'].dt.strftime('%Y-%m-%d')
BGLBNP.rename(columns={'Posting Date': 'Creation_date','Libellé':'Comment','Devise':'Currency'}, inplace=True)
# drop rows with null values in the 'Data' column
#fdrm = fdrm.dropna(subset=['Data'])

BGLBNP['Amount'] = BGLBNP['Montant'].astype(float)

# Do something with the DataFrame, e.g.
print(BGLBNP)



In [ ]:
BGLBNP.head()

### Summarizing/Adding Up the Data

In [ ]:
def rename_df_columns(df, column_mapping):
    return df.rename(columns=column_mapping)

In [ ]:
import pandas as pd
import sqlite3

# Example DataFrames (initialize with your actual data or leave undefined for testing)
# Uncomment and define the DataFrames you need
# rvlt = ...
# sodexo = ...
# traderep = ...
# satispay = ...
# fdrm = ...
# BGLBNP = ...

# List of required DataFrame names and columns
required_dfs = ['rvlt', 'sodexo', 'traderep', 'satispay', 'fdrm', 'BGLBNP']
columns = ['TRANSACTION_TYPE', 'DATE', 'DESCRIPTION', 'AMOUNT', 'CURRENCY', 'STATE', 'SUGGESTED_CATEGORY']

# Check for missing or empty DataFrames and remove them from the list
for df_name in required_dfs[:]:  # Use slicing to avoid modifying the list while iterating
    if df_name not in locals() or not isinstance(locals()[df_name], pd.DataFrame) or locals()[df_name].empty:
        print(f"Removing {df_name}: Not defined or empty.")
        required_dfs.remove(df_name)

# Example DataFrame standardization (only process existing DataFrames)
for df_name in required_dfs:
    df = locals()[df_name]  # Retrieve the DataFrame
    if df_name == 'rvlt':
        df.rename(columns={
            'Transaction_Type': 'TRANSACTION_TYPE',
            'Date': 'DATE',
            'Description': 'DESCRIPTION',
            'Amount': 'AMOUNT',
            'Currency': 'CURRENCY',
            'State': 'STATE'
        }, inplace=True)
    elif df_name == 'sodexo':
        df.rename(columns={
            'Date': 'DATE',
            'DESCRIPTION': 'DESCRIPTION',
            'Amount': 'AMOUNT'
        }, inplace=True)
    elif df_name == 'traderep':
        df.rename(columns={
            'Date': 'DATE',
            'DESCRIPTION': 'DESCRIPTION',
            'Amount': 'AMOUNT'
        }, inplace=True)
    elif df_name == 'satispay':
        df.rename(columns={
            'TRANSACTION_TYPE': 'TRANSACTION_TYPE',
            'date': 'DATE',
            'amount': 'AMOUNT',
            'currency': 'CURRENCY',
            'state': 'STATE',
            'DESCRIPTION': 'DESCRIPTION'
        }, inplace=True)
    elif df_name == 'fdrm':
        df.rename(columns={
            'DATE': 'DATE',
            'TRANSACTION_TYPE': 'TRANSACTION_TYPE',
            'DESCRIPTION': 'DESCRIPTION',
            'STATE': 'STATE',
            'SUGGESTED_CATEGORY': 'SUGGESTED_CATEGORY',
            'CURRENCY': 'CURRENCY',
            'Amount': 'AMOUNT'
        }, inplace=True)
    elif df_name == 'BGLBNP':
        df.rename(columns={
            'Creation_date': 'DATE',
            'Comment': 'DESCRIPTION',
            'Amount': 'AMOUNT',
            'Currency': 'CURRENCY'
        }, inplace=True)

    # Add missing columns
    for col in columns:
        if col not in df.columns:
            df[col] = None  # Fill missing columns with None/NaN

    # Ensure DATE is in datetime format and other types are consistent
    df['DATE'] = pd.to_datetime(df['DATE'], errors='coerce')  # Convert to datetime
    df['AMOUNT'] = pd.to_numeric(df['AMOUNT'], errors='coerce')  # Ensure numeric

# Create SQLite connection
conn = sqlite3.connect(":memory:")  # Use an in-memory SQLite database

# Write each DataFrame to SQLite
for df_name in required_dfs:
    locals()[df_name].to_sql(df_name, conn, index=False, if_exists="replace")

# Build SQL query dynamically
query_parts = []

if 'rvlt' in required_dfs:
    query_parts.append("""
        SELECT 'REVOLUT' AS ACCOUNT_NAME,
               TRANSACTION_TYPE,
               DATE,
               DESCRIPTION,
               AMOUNT,
               CURRENCY,
               STATE,
               '' AS SUGGESTED_CATEGORY
        FROM rvlt
    """)

if 'sodexo' in required_dfs:
    query_parts.append("""
        SELECT 'SODEXO' AS ACCOUNT_NAME,
               'CARD_PAYMENT' AS TRANSACTION_TYPE,
               DATE,
               DESCRIPTION,
               AMOUNT,
               'EUR' AS CURRENCY,
               'COMPLETED' AS STATE,
               '' AS SUGGESTED_CATEGORY
        FROM sodexo
    """)

if 'traderep' in required_dfs:
    query_parts.append("""
        SELECT 'TRADE_REP' AS ACCOUNT_NAME,
               'CARD_PAYMENT' AS TRANSACTION_TYPE,
               DATE,
               DESCRIPTION,
               AMOUNT,
               'EUR' AS CURRENCY,
               'COMPLETED' AS STATE,
               '' AS SUGGESTED_CATEGORY
        FROM traderep
    """)

if 'satispay' in required_dfs:
    query_parts.append("""
        SELECT 'SATISPAY' AS ACCOUNT_NAME,
               TRANSACTION_TYPE,
               DATE,
               DESCRIPTION,
               AMOUNT,
               CURRENCY,
               STATE,
               '' AS SUGGESTED_CATEGORY
        FROM satispay
    """)

if 'fdrm' in required_dfs:
    query_parts.append("""
        SELECT 'FIDEURAM' AS ACCOUNT_NAME,
               TRANSACTION_TYPE,
               DATE,
               DESCRIPTION,
               AMOUNT,
               CURRENCY,
               STATE,
               SUGGESTED_CATEGORY
        FROM fdrm
    """)

if 'BGLBNP' in required_dfs:
    query_parts.append("""
        SELECT 'BGLBNP' AS ACCOUNT_NAME,
               '' AS TRANSACTION_TYPE,
               DATE,
               DESCRIPTION,
               AMOUNT,
               CURRENCY,
               '' AS STATE,
               '' AS SUGGESTED_CATEGORY
        FROM BGLBNP
    """)

# Combine the query parts with UNION and ORDER BY
query = " UNION ".join(query_parts) + " ORDER BY DATE"

# Execute query and load the result into a pandas DataFrame
combined_result = pd.read_sql_query(query, conn)

# Close SQLite connection
conn.close()

# Display the result
print(combined_result)


In [ ]:
# Path to CSV file with month and year placeholders
csv_path = r'C:\Users\filip\Desktop\Monthly Transactions\AllTransactions_{}{}.csv'.format(month, year)

combined_result.to_csv(csv_path, index=False)